In [1]:
import os
import csv
import cv2
import numpy as np
import mediapipe as mp

###### Extraire les landmarks d'un image

In [4]:
def extract_landmarks(image, class_, hands):
    # Convertir l'image en format RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)
    # Extraction des dimensions de l'image
    h, w, c = image.shape
    # Traitement de l'image pour détecter les mains
    results = hands.process(image_rgb)
    # Initialisation du tableau pour stocker les coordonnées des landmarks
    landmarks_array = [class_]
    
    # Vérification de la présence de landmarks pour plusieurs mains dans les résultats
    if results.multi_hand_landmarks:
        # Boucle à travers les landmarks de chaque main détectée
        for hand_landmarks in results.multi_hand_landmarks:
            # Extraction des coordonnées des landmarks
            landmarks = hand_landmarks.landmark
            landmarks_row = []
            # Boucle à travers chaque landmark
            for landmark in landmarks:
                # Stockage des coordonnées des landmarks dans le tableau
                landmarks_row.extend([landmark.x*w, landmark.y*h, landmark.z*c])
            # Stockage des coordonnées des landmarks de la main dans le tableau principal
            landmarks_array.append(landmarks_row)
    
        # Traitement du nombre de mains détectées
        nom_hands = len(results.multi_hand_landmarks)
        
        # Ajout d'informations supplémentaires au tableau des landmarks en fonction du nombre de mains détectées
        if nom_hands == 1:
            landmarks_array[1].append(0)
            landmarks_array.append([0] * 63)
        elif nom_hands == 2:
            landmarks_array[1].append(1)
        
        # Retour des coordonnées des landmarks sous forme de tableau aplati
        return np.array([ [landmarks_array[0]]+landmarks_array[1]+landmarks_array[2] ]).flatten()
    else:
        # Si aucun landmark n'est détecté, retourne None
        return None

###### Génération de fichiers txt des landmarks des images de dataset

In [5]:
def generate_data(classes_folder):
    # Importation du modèle de détection des mains
    mpHands = mp.solutions.hands
    # Initialisation du détecteur de mains
    hands = mpHands.Hands()

    with open('hands_data.txt', 'w') as txtfile:
        # Création de l'en-tête du fichier texte
        header = 'class' + ''.join([f'\tld_1_{i}' for i in range(1, 64)]) + '\to_h' + ''.join([f'\tld_2_{i}' for i in range(1, 64)])+'\n'
        txtfile.write(header)

        # Parcours des répertoires des classes
        directory = os.path.join(os.getcwd(), classes_folder)
        dir_classes = os.listdir(directory)

        for class_ in dir_classes:
            class_dir_path = os.path.join(directory, class_)
            image_names = os.listdir(class_dir_path)

            # Parcours des images de chaque classe
            for image_name in image_names:
                image_path = os.path.join(class_dir_path, image_name)
                image = cv2.imread(image_path)

                # Extraction des landmarks de la main pour chaque image
                hand_landmarks = extract_landmarks(image, class_, hands)

                # Écriture des landmarks dans le fichier texte
                if hand_landmarks is not None:
                    hand_landmarks_str = [str(item) for item in hand_landmarks]
                    hand_landmarks_formatted = '\t'.join(hand_landmarks_str)
                    hand_landmarks_formatted+= '\n'
                    txtfile.write(hand_landmarks_formatted)


In [6]:
generate_data("DataSet")